In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from itertools import combinations
import itertools


In [2]:
csfbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\CSF_Biomarker2\csfbiomk2_20231117.csv"
csfbiomk2 = pd.read_csv(csfbiomk2_file_path)
csfbiomk2 = pd.DataFrame(csfbiomk2)

# right_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Volumetric_Analysis\rh_aparc_volume_20231117.csv"
# right_volume = pd.read_csv(right_volume_file_path)
# right_volume = pd.DataFrame(right_volume)


In [3]:
csfbiomk2.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,0.789,1.07,0.653,2.4,0.159,2.22,2.09,NaN,NaN,1.0
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print("Column Names:")
print(csfbiomk2.columns[2])

Column Names:
checkin_bin


In [5]:
#group them base on the value in the third column which indicates their level of playing
csfbiomk2_grouped = csfbiomk2.groupby(csfbiomk2.iloc[:, 2])

NFL_csfbiomk2_grouped = pd.DataFrame()
CP_csfbiomk2_grouped = pd.DataFrame()
HC_csfbiomk2_grouped = pd.DataFrame()


# group_name : 1, 2, 3   group_data: 
for group_name, group_data in csfbiomk2_grouped:
    if group_name == 1:
        NFL_csfbiomk2_grouped = pd.concat([NFL_csfbiomk2_grouped,group_data], ignore_index = True)
    if group_name == 2:
        CP_csfbiomk2_grouped = pd.concat([CP_csfbiomk2_grouped,group_data], ignore_index = True)
    if group_name == 3:
        HC_csfbiomk2_grouped = pd.concat([HC_csfbiomk2_grouped,group_data], ignore_index = True)
    
#print("DataFrame for NFL:")
#print(NFL_right_grouped.head())
NFL_csfbiomk2_grouped.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,1,1,2,3,15.0,22.0,8220.9,5421.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,1,1,2,5,16.0,20.0,9307.0,9307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,1,1,1,3,19.0,17.0,9866.7,6173.3,...,0.894,1.55,0.664,2.96,0.1000,1.74,1.77,3.74,219.0,0.0
4,1018,1,1,1,1,3,16.0,23.0,10635.9,7929.6,...,0.237,13.00,0.668,1.99,0.0636,9.98,1.78,NaN,NaN,NaN


In [6]:
index_of_chiiyrs_pg = NFL_csfbiomk2_grouped.columns.get_loc("chiiyrs_pg")
NFL_csfbiomk2_grouped.drop(columns=NFL_csfbiomk2_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
CP_csfbiomk2_grouped.drop(columns=CP_csfbiomk2_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
HC_csfbiomk2_grouped.drop(columns=HC_csfbiomk2_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)

NFL_csfbiomk2_grouped.head()


,subject_id,checkin_bin,Qalb,dxcte_albumin_csf_flag_dv,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,11.950965,0.0,36.656,2.57,0.241,0.643,15.4,15.6,...,0.894,1.55,0.664,2.96,0.1000,1.74,1.77,3.74,219.0,0.0
4,1018,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.237,13.00,0.668,1.99,0.0636,9.98,1.78,NaN,NaN,NaN


In [7]:
#combine all three classes
combined_csfbiomk2 = pd.concat([NFL_csfbiomk2_grouped, CP_csfbiomk2_grouped, HC_csfbiomk2_grouped], ignore_index=True)

combined_csfbiomk2.head()

,subject_id,checkin_bin,Qalb,dxcte_albumin_csf_flag_dv,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,11.950965,0.0,36.656,2.57,0.241,0.643,15.4,15.6,...,0.894,1.55,0.664,2.96,0.1000,1.74,1.77,3.74,219.0,0.0
4,1018,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.237,13.00,0.668,1.99,0.0636,9.98,1.78,NaN,NaN,NaN


In [8]:
# Remove NaNs
print("Size before:", combined_csfbiomk2.shape)
combined_csfbiomk2.dropna(inplace=True)
print("Size after:", combined_csfbiomk2.shape)

Size before: (236, 120)
Size after: (150, 120)


In [9]:
# Separate based on the level of professionalism
X = combined_csfbiomk2.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk2['checkin_bin']

In [10]:
X.head()

,Qalb,dxcte_albumin_csf_flag_dv,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
3,11.950965,0.0,36.656,2.570,0.2410,0.643,15.40,15.60,0.06730,3.680,...,0.894,1.550,0.664,2.96,0.1000,1.740,1.77,3.740,219.0,0.0
5,2.659185,0.0,8.056,4.590,0.1700,2.000,13.10,5.00,0.00441,4.630,...,1.900,0.599,0.768,0.00,0.0941,1.890,2.08,3.970,706.0,0.0
6,6.576438,0.0,23.858,0.432,0.0701,0.921,15.00,12.10,0.05660,1.850,...,1.800,0.463,0.893,8.88,0.0849,2.260,1.30,0.897,298.0,0.0
8,4.941153,0.0,17.003,1.150,0.1180,2.700,8.59,4.69,0.02770,0.990,...,2.050,4.740,0.671,7.89,0.0874,7.880,1.92,5.560,583.0,0.0
9,10.671813,0.0,32.485,0.944,0.3950,0.194,14.90,2.53,0.04490,0.463,...,0.122,3.500,0.550,1.04,0.0518,0.817,1.13,2.810,392.0,0.0


In [11]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [14]:
# RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# store evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

Accuracy: 0.5333333333333333, Precision: 0.3428571428571428, Recall: 0.5333333333333333, f1:0.41739130434782606


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
y_pred2  = model.predict(X_train_scaled)

accuracy_train = accuracy_score(y_train, y_pred2)
precision_bagging = precision_score(y_train, y_pred2, average='weighted')
recall_bagging = recall_score(y_train, y_pred2, average='weighted')
f1_bagging = f1_score(y_train, y_pred2, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")
# print(f"Accuracy: {accuracy}")


Accuracy: 0.5333333333333333, Precision: 0.3428571428571428, Recall: 0.5333333333333333, f1:0.41739130434782606


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [21]:
y_test.shape

(30,)

In [22]:
y_pred.shape

(30,)

In [32]:
# store evaluation metrics
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred, average = 'weighted')
recall = recall_score(y_train, y_pred, average = 'weighted')
f1 = f1_score(y_train, y_pred, average = 'weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

ValueError: Found input variables with inconsistent numbers of samples: [120, 30]

## Try Bagging


In [15]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                  n_estimators=10, random_state=42)


bagging_model.fit(X_train, y_train)
    
y_pred_bagging = bagging_model.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.5333333333333333, Precision: 0.3428571428571428, Recall: 0.5333333333333333, f1:0.41739130434782606


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
y_test

107    1
29     1
193    3
114    1
112    1
54     1
96     1
227    3
102    1
124    2
178    2
21     1
60     1
16     1
33     1
87     1
170    2
103    1
86     1
214    3
49     1
206    3
45     1
207    3
211    3
231    3
174    2
229    3
72     1
53     1
Name: checkin_bin, dtype: int64

## Try XGBoost

In [17]:
from xgboost import XGBClassifier


## Try XGBoost

In [18]:
X = combined_csfbiomk2.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk2['checkin_bin']

In [19]:
y

3      1
5      1
6      1
8      1
9      1
      ..
231    3
232    3
233    3
234    3
235    3
Name: checkin_bin, Length: 150, dtype: int64

In [20]:
# Since for XGBoost, the y values should start at 0, you need to do some justification
y = y-1

In [21]:
y

3      0
5      0
6      0
8      0
9      0
      ..
231    2
232    2
233    2
234    2
235    2
Name: checkin_bin, Length: 150, dtype: int64

In [22]:
# Do the splitting again to make sure that this adjustemnt is applied to your new X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [23]:
X_test.head()

,Qalb,dxcte_albumin_csf_flag_dv,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,0.670030,1.0,0.711872,0.553719,0.292086,0.013027,0.401042,0.469697,0.243660,0.000860,...,0.838096,0.033199,0.079372,0.296591,0.521615,0.021392,0.451300,0.173719,0.325083,0.0
1,0.957376,0.0,1.023711,0.276860,0.177403,0.010590,0.203451,0.113939,0.200269,0.015267,...,0.100337,0.064706,0.520074,0.007197,0.174473,0.044040,0.550463,0.212720,0.330033,0.0
2,0.141656,0.0,0.157827,0.268182,0.147671,0.010503,0.148112,0.187879,0.820453,0.042772,...,0.641185,0.090074,0.221043,0.325758,0.069028,0.012623,0.067871,0.021705,0.308581,0.0
3,0.196024,0.0,0.218861,0.064463,0.133513,0.016421,0.131836,0.324242,0.902746,0.012693,...,0.470529,0.077574,0.219197,0.025568,0.036467,0.070513,0.164830,0.016444,0.165017,0.0
4,0.262690,0.0,0.292630,0.114876,0.174572,0.043658,0.114909,0.151515,0.425451,0.067630,...,0.391327,0.023824,0.185971,0.060227,0.361318,0.007650,0.222124,0.169184,0.234323,0.0


In [24]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=100, random_state=42)

xgb_model.fit(X_train, y_train)
    
y_pred_xgb = xgb_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")



Accuracy: 0.5333333333333333, Precision: 0.3428571428571428, Recall: 0.5333333333333333, f1:0.41739130434782606
